1. Upload downloaded csv from [Full TMDB Movies Dataset 2024 (1M Movies) Kaggle Dataset](https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies) to Google Drive root folder.

2. Mount the Google Drive in Colab

3. Run the Script

In [ ]:
!pip install -U polars[gpu] --extra-index-url=https://pypi.nvidia.com
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu11
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12

In [2]:
import polars as pl

df = pl.scan_csv('/content/drive/MyDrive/TMDB_movie_dataset_v11.csv')

print(df)

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

Csv SCAN [/content/drive/MyDrive/TMDB_movie_dataset_v11.csv]
PROJECT */24 COLUMNS


In [7]:
%%time

# Group by production companies and calculate the max revenue
max_revenue_per_company = (df.group_by('production_companies')
                                    .agg(pl.col('revenue').max())
                                    .sort("revenue", descending=True)
                                    .collect())

CPU times: user 1.49 s, sys: 311 ms, total: 1.8 s
Wall time: 2.28 s


In [4]:
gpu_engine = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=True, # Fail loudly if we can't run on the GPU.
)

In [11]:
%%time

max_revenue_per_company = (df.group_by('production_companies')
                                    .agg(pl.col('revenue').max())
                                    .sort("revenue", descending=True)
                                    .collect(engine=gpu_engine))

CPU times: user 193 ms, sys: 298 ms, total: 491 ms
Wall time: 754 ms
